# Imports

In [1]:
import pandas as pd; import seaborn as sb; import warnings; import scipy; import re; 
import os; from analysis_helpers import *; import itertools; from scipy import stats
import random; import pandas as pd; import numpy as np; from sklearn import datasets, linear_model; 
from sklearn.linear_model import LinearRegression; import statsmodels.api as sm
from scipy import stats; from itertools import groupby; from operator import itemgetter;
import csv; import time
warnings.filterwarnings('ignore')
%matplotlib inline

# Import behavioral data 

Here, we ran a single presentation and memory run and recorded and time stamped the filename of each presented stimulus, its location, millisecond timings on window flips, etc, to verify the precision and accuracy of the data in our behavioral data files

In [18]:
with open('/Users/kirstenziman/Desktop/FINAL_EYE_2019_Sep_05/test_data/sustained_with_times_and_images_2.log') as file:
    lines = file.read().splitlines()
    lines = [[lines[x]] for x in range(0, len(lines))]
    lines.insert(0,["DATA"])

    with open('/Users/kirstenziman/Desktop/FINAL_EYE_2019_Sep_05/test_data/sustained_with_times_and_images_2.csv', 'w+') as csvfile:
        w = csv.writer(csvfile)
        w.writerows(lines)

# Log file data

In [77]:
log_file = pd.read_csv('/Users/kirstenziman/Desktop/FINAL_EYE_2019_Sep_05/test_data/sustained_with_times_and_images_2.csv')
log_file = log_file.reset_index()
log_file['TIME'],log_file['WARNING'],log_file['MESSAGE'] = log_file['DATA'].str.split('\t', 4).str
log_file = log_file.fillna('NAN VALUE')

In [78]:
log_file.head()

,index,DATA,TIME,WARNING,MESSAGE
0,0,4.7915 \tWARNING \twarning,4.7915,WARNING,warning
1,1,4.7916 \tEXP \texperiment,4.7916,EXP,experiment
2,2,4.7916 \tDATA \tdata,4.7916,DATA,data
3,3,4.7916 \tINFO \tinfo,4.7916,INFO,info
4,4,4.7916 \tINFO \tcurrent time: 1567699848.41,4.7916,INFO,current time: 1567699848.41


In [79]:
log_file['TIME'] = log_file['TIME'].str.replace(' ','')
log_file['TIME'] = log_file['TIME'].astype(float, errors='ignore')

In [80]:
p = log_file[(log_file['DATA'].str.contains('COMPOSITE'))
    & ((log_file['DATA'].str.contains('ON')))].melt(id_vars=['MESSAGE'], value_vars=['TIME'])

In [81]:
p['utc'] = [x[-13:] for x in list(p['value'])]

In [282]:
p.head()

,index,DATA,TIME,WARNING,MESSAGE,utc
1308,1308,64.4902 \tEXP \tWIN FLIP : STIMULI OFF ['CUED ...,64.4902,EXP,WIN FLIP : STIMULI OFF ['CUED COMPOSITE ../../...,1567699912.88
1329,1329,69.1254 \tEXP \tWIN FLIP : STIMULI OFF ['CUED ...,69.1254,EXP,WIN FLIP : STIMULI OFF ['CUED COMPOSITE ../../...,1567699917.52
1350,1350,74.1429 \tEXP \tWIN FLIP : STIMULI OFF ['CUED ...,74.1429,EXP,WIN FLIP : STIMULI OFF ['CUED COMPOSITE ../../...,1567699922.54
1371,1371,78.9600 \tEXP \tWIN FLIP : STIMULI OFF ['CUED ...,78.9600,EXP,WIN FLIP : STIMULI OFF ['CUED COMPOSITE ../../...,1567699927.35
1392,1392,83.6462 \tEXP \tWIN FLIP : STIMULI OFF ['CUED ...,83.6462,EXP,WIN FLIP : STIMULI OFF ['CUED COMPOSITE ../../...,1567699932.04


In [120]:
uncued = p[ (p['value'].str.contains('UNCUED')) & (p['value'].str.contains('ON')) & (~p['value'].str.contains('practice'))]
cued   = p[(~p['value'].str.contains('UNCUED')) & (p['value'].str.contains('ON')) & (~p['value'].str.contains('practice'))]
cued = cued.rename(columns={"value": "cued_onset"})
uncued = uncued.rename(columns={"value": "uncued_onset"})

cued   = cued.reset_index(drop=True)
uncued = uncued.reset_index(drop=True)

In [121]:
uncued['utc'][0] = uncued['utc'][0][1:]
uncued['utc'][1] = uncued['utc'][1][1:]

In [122]:
uncued

,MESSAGE,variable,uncued_onset,utc
0,NAN VALUE,TIME,ameUNCUEDCOMPOSITE../../stim/composite/0093396...,1567699909.8
1,NAN VALUE,TIME,ameUNCUEDCOMPOSITE../../stim/composite/0055994...,1567699914.5
2,NAN VALUE,TIME,ameUNCUEDCOMPOSITE../../stim/composite/0077494...,1567699919.52
3,NAN VALUE,TIME,ameUNCUEDCOMPOSITE../../stim/composite/0088496...,1567699924.34
4,NAN VALUE,TIME,ameUNCUEDCOMPOSITE../../stim/composite/0083594...,1567699929.02
5,NAN VALUE,TIME,ameUNCUEDCOMPOSITE../../stim/composite/0015493...,1567699933.64
6,NAN VALUE,TIME,ameUNCUEDCOMPOSITE../../stim/composite/0031394...,1567699938.39
7,NAN VALUE,TIME,ameUNCUEDCOMPOSITE../../stim/composite/0056894...,1567699943.24
8,NAN VALUE,TIME,ameUNCUEDCOMPOSITE../../stim/composite/0009093...,1567699947.94
9,NAN VALUE,TIME,ameUNCUEDCOMPOSITE../../stim/composite/0070596...,1567699952.55


In [124]:
cued['utc'][2] = cued['utc'][2][1:]

In [125]:
cued

,MESSAGE,variable,cued_onset,utc
0,NAN VALUE,TIME,ameCUEDCOMPOSITE../../stim/composite/001469606...,1567699909.78
1,NAN VALUE,TIME,ameCUEDCOMPOSITE../../stim/composite/009799606...,1567699914.48
2,NAN VALUE,TIME,ameCUEDCOMPOSITE../../stim/composite/008829605...,1567699919.5
3,NAN VALUE,TIME,ameCUEDCOMPOSITE../../stim/composite/010099606...,1567699924.31
4,NAN VALUE,TIME,ameCUEDCOMPOSITE../../stim/composite/007729412...,1567699929.01
5,NAN VALUE,TIME,ameCUEDCOMPOSITE../../stim/composite/009479606...,1567699933.62
6,NAN VALUE,TIME,ameCUEDCOMPOSITE../../stim/composite/007709412...,1567699938.38
7,NAN VALUE,TIME,ameCUEDCOMPOSITE../../stim/composite/009429606...,1567699943.23
8,NAN VALUE,TIME,ameCUEDCOMPOSITE../../stim/composite/009789606...,1567699947.92
9,NAN VALUE,TIME,ameCUEDCOMPOSITE../../stim/composite/007779412...,1567699952.54


In [126]:
uncued['utc'] = uncued['utc'].astype(float)
cued['utc']   = cued['utc'].astype(float)

In [127]:
uncued['uncued_onset'][0]

'ameUNCUEDCOMPOSITE../../stim/composite/00933960627fa_sunapwjkcxdaeqehfql.jpgON\\utc1567699909.8'

# average difference in onset times between Cued and Uncued Composite in a single test block (~1 frame)

In [128]:
x = list(uncued['utc']-cued['utc'])

In [129]:
print('Average number of seconds cued composite shows before uncued composite in a test run '+str(np.mean(x)))

Average number of seconds cued composite shows before uncued composite in a test run 0.01700000762939453


# Check timing and order of stimuli, log versus csv files

## verify stimulus orders same as in behavioral csv

### load behavioral csv

In [135]:
sub_list = []
data_dir = '/Users/kirstenziman/Desktop/FINAL_EYE_2019_Sep_05'

for sub_dir in os.listdir(data_dir):
    sub_list.append(add_level(sum_pd(data_dir + '/' + sub_dir)))
    print(sub_dir)
    
exp_raw = pd.concat(sub_list)
#exp1_raw.to_csv('../parsed_data/behavioral_data_sustained.csv')

test_data


# Cued composite images are in the same order in log file and csv

In [136]:
# uncued composite stimuli from log file
[x[36:71] for x in list(cued['cued_onset'])]

['/00146960620fb_sunaatpeezlljqaxlhj.',
 '/00979960627fa_sunbdycacjfaujxitbw.',
 '/00882960530fa_sunaknlirqjqiwbbeeg.',
 '/01009960627fa_sunbxzweleltykebgan.',
 '/00772941201fa_sunafpfpiiujzoesnqg.',
 '/00947960627fa_sunasmmcxdddtmtazwx.',
 '/00770941201fa_sunaflojskatpeqyzqp.',
 '/00942960627fa_sunarfdklgdnjgeqfjg.',
 '/00978960627fa_sunbddbivbptlrlauhy.',
 '/00777941205fa_sunagcinxkccdtrjmlv.']

In [137]:
exp_raw[exp_raw['Trial Type']=='Presentation']['Cued Composite']

0    00146960620fb_sunaatpeezlljqaxlhj.jpg
1    00979960627fa_sunbdycacjfaujxitbw.jpg
2    00882960530fa_sunaknlirqjqiwbbeeg.jpg
3    01009960627fa_sunbxzweleltykebgan.jpg
4    00772941201fa_sunafpfpiiujzoesnqg.jpg
5    00947960627fa_sunasmmcxdddtmtazwx.jpg
6    00770941201fa_sunaflojskatpeqyzqp.jpg
7    00942960627fa_sunarfdklgdnjgeqfjg.jpg
8    00978960627fa_sunbddbivbptlrlauhy.jpg
9    00777941205fa_sunagcinxkccdtrjmlv.jpg
Name: Cued Composite, dtype: object

# Uncued composite images are in the same order in log file and csv

In [138]:
[x[36:71] for x in list(uncued['uncued_onset'])]

['te/00933960627fa_sunapwjkcxdaeqehfq',
 'te/00559940519fa_sunacjkmnpawfzbyva',
 'te/00774941201fa_sunafvtpfczkzopbgu',
 'te/00884960530fa_sunakodkcslgqhqlgw',
 'te/00835940307fa_sunaijsqpvodplbwxk',
 'te/00154931230fa_sunaauueuhhvklldji',
 'te/00313940422fa_sunabwjvmoaddlyxfj',
 'te/00568940928fa_sunacsdhbbsybcyxne',
 'te/00090931230fa_sunaahedjlmtqekgyv',
 'te/00705960530fa_sunaduslxfzlbxllln']

In [139]:
exp_raw[exp_raw['Trial Type']=='Presentation']['Uncued Composite']

0    00933960627fa_sunapwjkcxdaeqehfql.jpg
1    00559940519fa_sunacjkmnpawfzbyvae.jpg
2    00774941201fa_sunafvtpfczkzopbgue.jpg
3    00884960530fa_sunakodkcslgqhqlgwl.jpg
4    00835940307fa_sunaijsqpvodplbwxkf.jpg
5    00154931230fa_sunaauueuhhvklldjik.jpg
6    00313940422fa_sunabwjvmoaddlyxfjr.jpg
7    00568940928fa_sunacsdhbbsybcyxnel.jpg
8    00090931230fa_sunaahedjlmtqekgyvj.jpg
9    00705960530fa_sunaduslxfzlbxlllnz.jpg
Name: Uncued Composite, dtype: object

# Onset times differ by fractions of a second

### csv onset times (same for cued and uncued composite)

In [235]:
d = {}

In [284]:
# aggregate onset times (two onsets in log file, one for cued composite and one for uncued; csv has only one onset time assigned to both)
d['csv_cued_onset'] = list(exp_raw[exp_raw['Trial Type']=='Presentation']['Stimulus Onset'])
d['log_cued_onset'] = list(cued['utc'])
d['log_uncued_onset'] = list(uncued['utc'])

# aggregate stim end times
p = log_file[(log_file['DATA'].str.contains('OFF')) & ((log_file['DATA'].str.contains('COMPOSITE'))) & ((~log_file['DATA'].str.contains('practice')))]
p['utc'] = [x[-13:] for x in list(p['DATA'])]
d['log_composite_off'] = [float(x) for x in p['utc']]
d['csv_composite_off'] = list(exp_raw[exp_raw['Trial Type']=='Presentation']['Stimulus End'])

### check time diffs

In [285]:
d = pd.DataFrame(d)
d

,csv_cued_onset,log_cued_onset,log_uncued_onset,log_composite_off,csv_composite_off,cue_on_diff,uncue_on_diff,composite_off_diff
0,1.567700e+09,1.567700e+09,1.567700e+09,1.567700e+09,1.567700e+09,0.034996,0.014996,-0.013137
1,1.567700e+09,1.567700e+09,1.567700e+09,1.567700e+09,1.567700e+09,0.036890,0.016890,-0.017955
2,1.567700e+09,1.567700e+09,1.567700e+09,1.567700e+09,1.567700e+09,0.034418,0.014418,-0.019700
3,1.567700e+09,1.567700e+09,1.567700e+09,1.567700e+09,1.567700e+09,0.043570,0.013570,-0.013334
4,1.567700e+09,1.567700e+09,1.567700e+09,1.567700e+09,1.567700e+09,0.028816,0.018816,-0.018223
5,1.567700e+09,1.567700e+09,1.567700e+09,1.567700e+09,1.567700e+09,0.036195,0.016195,-0.020062
6,1.567700e+09,1.567700e+09,1.567700e+09,1.567700e+09,1.567700e+09,0.027392,0.017392,-0.018645
7,1.567700e+09,1.567700e+09,1.567700e+09,1.567700e+09,1.567700e+09,0.029677,0.019677,-0.016488
8,1.567700e+09,1.567700e+09,1.567700e+09,1.567700e+09,1.567700e+09,0.040956,0.020956,-0.014900
9,1.567700e+09,1.567700e+09,1.567700e+09,1.567700e+09,1.567700e+09,0.022165,0.012165,-0.012976


In [286]:
d['cue_on_diff'] = d['csv_cued_onset']-d['log_cued_onset']
d['uncue_on_diff'] = d['csv_cued_onset']-d['log_uncued_onset']

print('mean onset difference between csv and log file, cued composites: '+str(d['cue_on_diff'].mean()))
print('^^ this is approximately two frames')
print()

print('mean onset difference between csv andn log file, uncued composites: '+str(d['uncue_on_diff'].mean()))
print('^^ this is approximately one frame')

mean onset difference between csv and log file, cued composites: 0.033507490158081056
^^ this is approximately two frames

mean onset difference between csv andn log file, uncued composites: 0.016507482528686522
^^ this is approximately one frame


In [287]:
d['composite_off_diff'] = d['csv_composite_off']-d['log_composite_off']

print('mean end time difference between csv and log file, for both composites: '+str(d['composite_off_diff'].mean()))
print('^^ this is approximately one frame')

mean end time difference between csv and log file, for both composites: -0.01654200553894043
^^ this is approximately one frame


# Check Memory Images

Memory images from log

In [288]:
[x[-45:-20] for x in list(log_file[(log_file['DATA'].str.contains('MEMORY')) & (log_file['DATA'].str.contains('ON')) & (~log_file['DATA'].str.contains('practice'))]['DATA'])]

['single/00762941201fa.jpg ',
 '/single/00194940128fb.jpg',
 'single/00313940422fa.jpg ',
 'single/01009960627fa.jpg ',
 '/sunaauueuhhvklldjik.jpg ',
 'single/00882960530fa.jpg ',
 'single/00705960530fa.jpg ',
 'single/00144931230fb.jpg ',
 '/sunafgzgbisbopcyslr.jpg ',
 'e/sunawotxrevnqvamsfp.jpg',
 'single/00154931230fa.jpg ',
 '/sunagcinxkccdtrjmlv.jpg ',
 'single/00743941201fa.jpg ',
 '/sunaknlirqjqiwbbeeg.jpg ',
 '/sunaazezcsunmhttzxz.jpg ',
 'single/00505960627fa.jpg ',
 'single/00777941205fa.jpg ',
 '/sunbdycacjfaujxitbw.jpg ',
 'single/00961960627fa.jpg ',
 'single/00884960530fa.jpg ',
 '/sunaathajvxrgptwhoj.jpg ',
 '/single/00770941201fa.jpg',
 'single/00557940519fa.jpg ',
 '/sunaflojskatpeqyzqp.jpg ',
 '/sunacacflqoyxhsqzhe.jpg ',
 'single/00507940519fa.jpg ',
 'single/00198940128fb.jpg ',
 '/single/00827940307fa.jpg',
 '/sunabzuqsrgcsonqlkz.jpg ',
 'single/00559940519fa.jpg ',
 '/sunahtfxgsrmkiulnwk.jpg ',
 '/sunakodkcslgqhqlgwl.jpg ',
 '/sunachtuarilwyqnvjx.jpg ',
 '/sunabwj

Memory images from csv

In [289]:
list(exp_raw[exp_raw['Trial Type']=='Memory']['Memory Image'])

['00762941201fa.jpg',
 '00194940128fb.jpg',
 '00313940422fa.jpg',
 '01009960627fa.jpg',
 'sunaauueuhhvklldjik.jpg',
 '00882960530fa.jpg',
 '00705960530fa.jpg',
 '00144931230fb.jpg',
 'sunafgzgbisbopcyslr.jpg',
 'sunawotxrevnqvamsfp.jpg',
 '00154931230fa.jpg',
 'sunagcinxkccdtrjmlv.jpg',
 '00743941201fa.jpg',
 'sunaknlirqjqiwbbeeg.jpg',
 'sunaazezcsunmhttzxz.jpg',
 '00505960627fa.jpg',
 '00777941205fa.jpg',
 'sunbdycacjfaujxitbw.jpg',
 '00961960627fa.jpg',
 '00884960530fa.jpg',
 'sunaathajvxrgptwhoj.jpg',
 '00770941201fa.jpg',
 '00557940519fa.jpg',
 'sunaflojskatpeqyzqp.jpg',
 'sunacacflqoyxhsqzhe.jpg',
 '00507940519fa.jpg',
 '00198940128fb.jpg',
 '00827940307fa.jpg',
 'sunabzuqsrgcsonqlkz.jpg',
 '00559940519fa.jpg',
 'sunahtfxgsrmkiulnwk.jpg',
 'sunakodkcslgqhqlgwl.jpg',
 'sunachtuarilwyqnvjx.jpg',
 'sunabwjvmoaddlyxfjr.jpg',
 '00979960627fa.jpg',
 'sunbxzweleltykebgan.jpg',
 'sunaeoghzrjarigmwua.jpg',
 'sunaayehfdqcyxggsej.jpg',
 'sunaduslxfzlbxlllnz.jpg',
 'sunacjkmnpawfzbyvae.jpg']

Onset times from log

In [298]:
d2={}

In [299]:
onset_list = [x[-13:] for x in list(log_file[(log_file['DATA'].str.contains('MEMORY')) & (log_file['DATA'].str.contains('ON')) & (~log_file['DATA'].str.contains('practice'))]['DATA'])]
d2['memory_onset_log'] = [float(x.replace("c", "")) for x in onset_list]
#onset_list

Onset times from csv

In [300]:
d2['memory_onset_csv'] = list(exp_raw[exp_raw['Trial Type']=='Memory']['Stimulus Onset'])

Aggregate end times from log and csv

In [309]:
d2['memory_off_log']=[x[-13:] for x in list(log_file[(log_file['DATA'].str.contains('MEMORY')) & (log_file['DATA'].str.contains('OFF')) & (~log_file['DATA'].str.contains('practice'))]['DATA'])]
d2['memory_off_csv'] = list(exp_raw[exp_raw['Trial Type']=='Memory']['Stimulus End'])

In [310]:
d2 = pd.DataFrame(d2)
d2.head()

,memory_onset_log,memory_onset_csv,memory_off_log,memory_off_csv,mem_onset_diff,mem_off_diff
0,1.567700e+09,1.567700e+09,1567699961.68,1.567700e+09,0.041525,-0.248735
1,1.567700e+09,1.567700e+09,1567699965.06,1.567700e+09,0.033353,-0.244155
2,1.567700e+09,1.567700e+09,1567699968.44,1.567700e+09,0.036564,-0.239359
3,1.567700e+09,1.567700e+09,1567699971.82,1.567700e+09,0.034017,-0.251225
4,1.567700e+09,1.567700e+09,1567699975.22,1.567700e+09,0.029936,-0.266925


In [311]:
d2['mem_onset_diff'] = d2['memory_onset_csv'] - d2['memory_onset_log']

In [318]:
print('Average difference in memory image onset time between csv and log: '+str(d2['mem_onset_diff'].mean()))

Average difference in memory image onset time between csv and log: 0.029798215627670287


In [313]:
d2['memory_off_log'] = pd.to_numeric(d2['memory_off_log'])
d2['mem_off_diff'] = d2['memory_off_csv'] - d2['memory_off_log']

In [320]:
print('Average difference in end time between csv and log: '+str(d2['mem_off_diff'].mean()))

Average difference in end time between csv and log: -0.2511367857456207


this longer lag in end time is likely due to the amount of time taken to extract the rating from the rating scale. Can time this, to check, and also move the rating scale data pulling code to after the stimulus has been pulled from the screen